In [13]:
# external api
import fundamentalanalysis as fa
import yfinance as yf

# data analytic 
import numpy as np
import pandas as pd

# additional liberies
import datetime
import statistics as st

In [14]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

api_keys

In [15]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'  #job.chap@icloud.com
# api_key = 'c81352430e2fe3c941faf0814227562b'  #jobs.chaps@gmail.com
# api_key = 'a765d11740cccfb61177da0ad8699d1e'  #job.chap@gmail.com

In [16]:
''' Тикеры '''
tick = 'F'  # поменял название переменно ticker на tick, для настройки по одной акции

''' для фильтрации'''
filt_start_date = '2000'
filt_finish_date = '2022'


''' время '''
stime=f'{str(int(filt_start_date)-1)}-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [19]:
''' получаем df с key_metrics
'''
key_metrics = fa.key_metrics(tick, api_key, period='annual')
df_key_metr = key_metrics.T


In [20]:
''' получаем df с income_statement
'''
income_statement = fa.income_statement(ticker, api_key, period='annual')
df_income_statement = income_statement.T

In [21]:
''' получаем df с balance_sheet_statement
'''
balance_sheet_statement = fa.balance_sheet_statement(tick, api_key, period='annual')
df_balance_sheet_statement = balance_sheet_statement.T

In [22]:
''' получаем df с cash_flow_statement
'''
cash_flow_statement = fa.cash_flow_statement(tick, api_key, period='annual')
df_cash_flow_statement = cash_flow_statement.T

In [23]:
''' получаем df с financial_ratios
'''
financial_ratios = fa.financial_ratios(tick, api_key, period='annual')
df_financial_ratios = financial_ratios.T

In [24]:
''' получаем df с financial_statement_growth
''' 

financial_statement_growth = fa.financial_statement_growth(tick, api_key, period='annual')
df_financial_statement_growth = financial_statement_growth.T


In [29]:
'''
'''
df_all_fa_columns = pd.concat([df_key_metr,
                               df_income_statement,
                               df_balance_sheet_statement,
                               df_cash_flow_statement,
                               df_financial_ratios,
                               df_financial_statement_growth], axis=1) #.loc[:,~pd.concat([df1, df2], axis=1).columns.duplicated()]

df_all_fa_columns.shape

(38, 274)

In [30]:
''' убрал дубликаты
'''
df_all_fa_columns = df_all_fa_columns.loc[:,~pd.concat([df_key_metr,
                                                        df_income_statement,
                                                        df_balance_sheet_statement,
                                                        df_cash_flow_statement,
                                                        df_financial_ratios,
                                                        df_financial_statement_growth], axis=1).columns.duplicated()]

df_all_fa_columns.shape

(38, 241)

In [32]:
''' удаляются не используемые колонки 
'''
df_all_fa_columns = df_all_fa_columns .drop(['period', 'reportedCurrency', 'cik', 'fillingDate', 'calendarYear', 'link', 'finalLink'], axis=1)

df_all_fa_columns.shape

(38, 234)

In [44]:
df_all_fa_columns.tail(3)

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,acceptedDate,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,cashFlowToDebtRatio,companyEquityMultiplier,fixedAssetTurnover,assetTurnover,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,enterpriseValueMultiple,priceFairValue,revenueGrowth,grossProfitGrowth,ebitgrowth,operatingIncomeGrowth,netIncomeGrowth,epsgrowth,epsdilute

In [34]:
def change_index_plus_one(df):
    ''' добавляем цифру к году, чтобы видень данные как начало года, а не как факт 
    '''
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        df_index[i] = str(int(df_index[i])+1)
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

df_update_year = change_index_plus_one(df_all_fa_columns)

In [37]:
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ''' получаем цены из yahoo_finance
    '''
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z



In [38]:
def change_index_yahoo(df):
    ''' сокращаем индекс их dataframe yahoo
    '''
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        t = df_index[i].date()
        df_index[i] = t.strftime('%Y-%m-%d')
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

In [41]:
''' получаем датафрейм для тикера и snp с корректной датой
'''

df_ticker = change_index_yahoo(get_data_from_ticker(tick, stime, None, period))  # цена нужной акции + измененный индекс
df_snp = change_index_yahoo(get_data_from_ticker('^GSPC', stime, None, period))  # цены SNP + измененный индекс

In [43]:
def get_years_prices(df, list_dates):
    ''' из датафрейма цен получает цены на начало года
    '''
    
    year_prices = []
    for i in range(len(list_dates)):
        x = int(list_dates[i])
        x = str(x)
        r = df.loc[f'{x}-01-01']
        r = float(r)
        year_prices.append(r)

    return year_prices

# извлекаем нужный период дат 
index_year = df_update_year.index.to_list()

# формируем таблицы с цена на начало года
df_ticker_years = get_years_prices(df_ticker, index_year)
df_snp_years = get_years_prices(df_snp, index_year)

KeyError: '1998-01-01'

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,acceptedDate,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,cashFlowToDebtRatio,companyEquityMultiplier,fixedAssetTurnover,assetTurnover,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,enterpriseValueMultiple,priceFairValue,revenueGrowth,grossProfitGrowth,ebitgrowth,operatingIncomeGrowth,netIncomeGrowth,epsgrowth,epsdilute

In [36]:
'''
'''

df_update_year['acceptedDate']

Date_index
2023    2023-02-02 19:39:34
2022    2022-02-03 18:24:07
2021    2021-02-04 18:58:09
2020    2020-02-05 12:01:22
2019    2019-02-21 16:10:10
2018    2018-02-08 11:41:16
2017    2017-02-09 08:44:28
2016    2016-02-11 08:13:33
2015    2015-02-12 19:00:00
2014    2014-02-17 19:00:00
2013    2013-02-19 07:12:51
2012    2012-02-21 15:45:59
2011    2011-02-27 19:00:00
2010    2010-02-25 14:03:27
2009    2009-02-26 06:17:07
2008    2008-02-26 19:00:00
2007    2007-02-27 19:00:00
2006    2006-02-28 19:00:00
2005    2005-03-09 19:00:00
2004    2004-03-11 19:00:00
2003    2002-12-30 19:00:00
2002    2002-03-28 00:00:00
2001    2001-03-22 00:00:00
2000    1999-12-30 19:00:00
1999    1998-12-30 19:00:00
1998    1997-12-30 19:00:00
1997    1996-12-30 19:00:00
1996    1995-12-30 19:00:00
1995    1995-03-16 00:00:00
1994    1993-12-30 19:00:00
1993    1992-12-30 19:00:00
1992    1991-12-30 19:00:00
1991    1990-12-30 19:00:00
1990    1989-12-30 19:00:00
1989    1988-12-30 19:00:00
1988    1